In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd

In [2]:
world = gpd.read_file('data/world-countries.json')
world.columns =  ['id', 'country', 'geometry']

population_df = pd.read_csv('data/population_prospects.csv')
population_df = population_df.sort_values(by="country")
countries_to_replace = {"Bahamas": "The Bahamas",
                        "Bolivia (Plurinational State of)": "Bolivia",
                        "Brunei Darussalam": "Brunei",
                        "Congo": "Republic of the Congo",
                        "Czechia": "Czech Republic",
                        "Cote d'Ivoire": "Ivory Coast",
                        "French Guiana": "Equatorial Guinea",
                        "Falkland Islands (Malvinas)": "Falkland Islands",
                        "Guinea-Bissau": "Guinea Bissau",
                        "Iran (Islamic Republic of)": "Iran",
                        "Republic of Korea": "South Korea",
                        "Dem. People's Republic of Korea": "North Korea",
                        "Lao People's Democratic Republic": "Laos",
                        "Republic of Moldova": "Moldova",
                        "North Macedonia": "Macedonia",
                        "Russian Federation": "Russia",
                        "Serbia": "Republic of Serbia",
                        "Syrian Arab Republic": "Syria",
                        "Timor-Leste": "East Timor",
                        "China, Taiwan Province of China": "Taiwan",
                        "Venezuela (Bolivarian Republic of)": "Venezuela",
                        "Viet Nam": "Vietnam"}
population_df["country"].replace(countries_to_replace, inplace=True)

D:\programs\Anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [3]:
df_population_2100 = population_df[(population_df["year"] == 2100) & (population_df["variant"] == "Medium variant")]
df_population_2001 = population_df[(population_df["year"] == 2001) & (population_df["variant"] == "Estimates")]
df_population_2100['difference'] = 100*(df_population_2100.population.values - df_population_2001.population.values)/df_population_2001.population.values

<ipython-input-3-c6dc184545f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population_2100['difference'] = 100*(df_population_2100.population.values - df_population_2001.population.values)/df_population_2001.population.values


# 1.
Основна суть даного графіку показати тенденцію в яких країнах збільшиться,а яких зменшиться, з цим він успішно справляється.
Основним недоліком є те, що неможливо порівнювати зміну окрім як збільшення або зменшення.

In [ ]:
alt.Chart(world).transform_lookup(lookup = 'country',
    from_ = alt.LookupData(data = df_population_2100, 
                           key = 'country',
                           fields=['difference', 'country'])
).project(type = 'equalEarth').transform_calculate(
  population_difference="datum.difference < 0.0 ? 'Decreased' : 'Increased'"
).mark_geoshape(stroke='gray').encode(tooltip = alt.Tooltip('country:N'),
        color = alt.Color("population_difference:N",
                      scale=alt.Scale(range = ['#D50032', '#00B52A']),
                      legend=alt.Legend(title=None)),
).properties(width=900, height=500, padding=25)

2.
Рішення яке чітко показує % приросту населення або ж його падіння.
Особисто на мою думку, оптимальний варіант.
Єдиний недолік, важко говорити про очікувану кілкість.


In [ ]:
domain = [df_population_2100.min()['difference'],-40, -30, -20, -10, 0, 100, 300, 600, 900, df_population_2100.max()['difference']]
alt.Chart(world).transform_lookup(
    lookup = 'country',
    from_ = alt.LookupData(data = df_population_2100,
                           key = 'country',
                           fields=['difference', 'country'])
).project(type = 'equalEarth'
).mark_geoshape(stroke='gray').encode(
    color = alt.Color('difference:Q',
                      scale=alt.Scale(domain=domain, scheme = 'redyellowgreen'),
                      title='Percentage difference'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('difference:N')
    ]
).properties(title="Population percentage change 2001 vs 2100",
             width = 800, height = 500, background = '#F9F9F9', padding = 25)

In [ ]:
3.
Хороший для оцінювання кількості, проте доволі важко розібратись в самій інформації.
Дано кількість людей в кожній країні і неможливо з чимось порівняти.


In [ ]:
alt.Chart(world).transform_lookup(
    lookup = 'country',
    from_ = alt.LookupData(data = df_population_2100,
                           key = 'country',
                           fields=['population', 'country'])
).project(type = 'equalEarth'
).mark_geoshape(stroke='gray').encode(
    color = alt.Color('population:Q',
                      scale=alt.Scale(scheme = 'bluegreen'),
                      bin = alt.Bin(step = 100000),
                      title='Population 2100'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('population:N')
    ]
).properties(title="Population 2100",
             width = 800, height = 500, background = '#F9F9F9', padding = 25)



